# Model Training for Real Time Communication through AI for Specially Abled
### Loading the Dataset & Image Data Generation

In [4]:
pwd

'/home/wsuser/work'

In [3]:
!pip install tensorflow==2.7.1

     |████████████████████████████████| 495.2 MB 32 kB/s s eta 0:00:01                      | 40.0 MB 17.3 MB/s eta 0:00:27ta 0:00:01
     |████████████████████████████████| 463 kB 63.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 76.6 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.1
    Uninstalling tensorflow-2.9.1:
      Successfully uninstalled tensorflow-2.9.1


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-05-28 10:54:56.349463: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow


In [6]:
# Training Datagen
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
# Testing Datagen
test_datagen = ImageDataGenerator(rescale=1/255)

In [18]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9e2ebbcc4db04f3fb5d87e8fa4800e36 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GN91c7sTtlR1DYfMzZqIUwGSIkATCCriPItR-s81P82-',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_9e2ebbcc4db04f3fb5d87e8fa4800e36.get_object(Bucket='communicationmodeltraining-donotdelete-pr-xpzs67frbbb7s3', Key='Communication_Dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [20]:
# Unzip the Dataset Zip File
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [24]:
%%bash
ls Communication_Dataset

test_set
training_set


In [25]:
# Training Dataset
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Communication_Dataset/training_set',target_size=(64,64), class_mode='categorical',batch_size=900)
# Testing Dataset
x_test=test_datagen.flow_from_directory(r'/home/wsuser/work/Communication_Dataset/test_set',target_size=(64,64), class_mode='categorical',batch_size=900)

Found 27000 images belonging to 9 classes.
Found 25737 images belonging to 9 classes.


In [26]:
print("Len x-train : ", len(x_train))
print("Len x-test : ", len(x_test))

Len x-train :  30
Len x-test :  29


In [27]:
# The Class Indices in Training Dataset
x_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

### Model Creation

In [28]:
# Importing Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

In [29]:
# Creating Model
model=Sequential()

2022-05-28 11:09:39.431060: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-05-28 11:09:39.431118: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [30]:
# Adding Layers
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Adding Hidden Layers
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))

# Adding Output Layer
model.add(Dense(9,activation='softmax'))

In [31]:
# Compiling the Model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [32]:
# Fitting the Model Generator
model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))

/tmp/wsuser/ipykernel_156/1042518445.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))


Epoch 1/10
30/30 [==============================] - 128s 4s/step - loss: 2.4043 - accuracy: 0.1420 - val_loss: 2.0750 - val_accuracy: 0.2559
Epoch 2/10
30/30 [==============================] - 130s 4s/step - loss: 1.9003 - accuracy: 0.3433 - val_loss: 1.6226 - val_accuracy: 0.4541
Epoch 3/10
30/30 [==============================] - 128s 4s/step - loss: 1.5307 - accuracy: 0.4587 - val_loss: 1.3421 - val_accuracy: 0.5421
Epoch 4/10
30/30 [==============================] - 129s 4s/step - loss: 1.2714 - accuracy: 0.5461 - val_loss: 1.0878 - val_accuracy: 0.6286
Epoch 5/10
30/30 [==============================] - 128s 4s/step - loss: 1.0807 - accuracy: 0.6205 - val_loss: 0.8924 - val_accuracy: 0.6984
Epoch 6/10
30/30 [==============================] - 128s 4s/step - loss: 0.9280 - accuracy: 0.6797 - val_loss: 0.7627 - val_accuracy: 0.7371
Epoch 7/10
30/30 [==============================] - 126s 4s/step - loss: 0.8069 - accuracy: 0.7210 - val_loss: 0.6570 - val_accuracy: 0.7660
Epoch 8/10
30

### Saving the Model

In [33]:
model.save('IBM_Communication_Model.h5')
# Current accuracy is 0.8154

In [34]:
# Convert the Saved Model to a Tar Compressed Format
!tar -zcvf IBM_TrainedModel.tgz IBM_Communication_Model.h5

IBM_Communication_Model.h5


In [35]:
%%bash
ls -ll

total 203152
drwxrwx--- 4 wsuser wscommon      4096 May 28 11:05 Communication_Dataset
-rw-rw---- 1 wsuser wscommon 111324760 May 28 11:33 IBM_Communication_Model.h5
-rw-rw---- 1 wsuser wscommon  96688252 May 28 11:34 IBM_TrainedModel.tgz


### Watson Machine Learning

In [36]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 23.1 MB/s eta 0:00:01


In [37]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "mNVF7E95G-awR213njShj1GiUfN-1SpPq-ko8Wx7na1-"
}

client = APIClient(wml_credentials)

### Save to Deployment Space

In [38]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [39]:
space_uid = guid_from_space_name(client, 'communication_model_deployment')
print("Space UID : ", space_uid)

Space UID :  21c15ae0-ee26-497d-b615-eb30ef2e16fe


In [40]:
client.set.default_space(space_uid)

'SUCCESS'

In [41]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [62]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [ ]:
model_details = client.repository.store_model(model='IBM_TrainedModel.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_uid(model_details)

In [66]:
model_id

'cefca265-2301-4620-897a-9c80d6ff7f1a'